<a href="https://colab.research.google.com/github/Salma9507/Lab-8-Plant-Disease-Classification/blob/main/Lab08_Final_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab08: Plant Disease Classification


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import save_model, load_model
from sklearn.model_selection import ParameterGrid
import numpy as np
import os


In [8]:
!pip install -q gdown
!gdown https://drive.google.com/uc?id=1MCQ2ldiKZUeVM1rVw1gPlBaX43AJB3R0  # Train
!gdown https://drive.google.com/uc?id=1yqvfEVeb0IAutxZK83_wUoUWm5apYSF8  # Test

import zipfile

with zipfile.ZipFile('plant_leave_diseases_train.zip', 'r') as zip_file:
    zip_file.extractall()

with zipfile.ZipFile('plant_leave_diseases_test.zip', 'r') as zip_file:
    zip_file.extractall()


Downloading...
From (original): https://drive.google.com/uc?id=1MCQ2ldiKZUeVM1rVw1gPlBaX43AJB3R0
From (redirected): https://drive.google.com/uc?id=1MCQ2ldiKZUeVM1rVw1gPlBaX43AJB3R0&confirm=t&uuid=8db5986e-8677-4ecb-8836-6b1dae047876
To: /content/plant_leave_diseases_train.zip
100% 682M/682M [00:08<00:00, 77.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1yqvfEVeb0IAutxZK83_wUoUWm5apYSF8
From (redirected): https://drive.google.com/uc?id=1yqvfEVeb0IAutxZK83_wUoUWm5apYSF8&confirm=t&uuid=decd6a16-659a-4e8c-97d4-6caebdd825cd
To: /content/plant_leave_diseases_test.zip
100% 170M/170M [00:02<00:00, 73.9MB/s]


In [9]:
train_data_dir = 'plant_leave_diseases_train'
img_size = (256, 256)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


In [10]:
param_grid = {
    'activation': ['relu', 'tanh'],
    'optimizer': ['adam', 'rmsprop'],
    'dense_units': [64, 128],
    'dropout_rate': [0.0, 0.5]
}

def create_model(activation='relu', optimizer='adam', dense_units=128, dropout_rate=0.0, num_classes=15):
    model = Sequential([
        Input(shape=(img_size[0], img_size[1], 3)),
        Conv2D(32, (3, 3), activation=activation),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation=activation),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation=activation),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(dense_units, activation=activation),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
best_val_acc = 0
best_params = None
os.makedirs("saved_models", exist_ok=True)

for params in ParameterGrid(param_grid):
    print(f"Testing: {params}")
    train_gen = datagen.flow_from_directory(
        train_data_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    val_gen = datagen.flow_from_directory(
        train_data_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    num_classes = train_gen.num_classes
    model = create_model(
        activation=params['activation'],
        optimizer=params['optimizer'],
        dense_units=params['dense_units'],
        dropout_rate=params['dropout_rate'],
        num_classes=num_classes
    )
    history = model.fit(train_gen, validation_data=val_gen, epochs=2)

    val_acc = history.history['val_accuracy'][-1]
    print(f"Validation accuracy: {val_acc:.4f}")

    model_name = (
        f"model_act-{params['activation']}_opt-{params['optimizer']}_"
        f"units-{params['dense_units']}_drop-{params['dropout_rate']}_"
        f"acc-{val_acc:.4f}.h5"
    )
    model_path = os.path.join("saved_models", model_name)
    save_model(model, model_path)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_params = params
        best_model_path = model_path

print("\nBest parameters:")
print(best_params)
print(f"Best validation accuracy: {best_val_acc:.4f}")

print(f"Best model saved at: {best_model_path}")


Testing: {'activation': 'relu', 'dense_units': 64, 'dropout_rate': 0.0, 'optimizer': 'adam'}
Found 34756 images belonging to 38 classes.
Found 8673 images belonging to 38 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 4758s 4s/step - accuracy: 0.4369 - loss: 2.1449 - val_accuracy: 0.7928 - val_loss: 0.6808
Epoch 2/2
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 4797s 4s/step - accuracy: 0.8206 - loss: 0.5824 - val_accuracy: 0.8426 - val_loss: 0.5003


Validation accuracy: 0.8426
Testing: {'activation': 'relu', 'dense_units': 64, 'dropout_rate': 0.0, 'optimizer': 'rmsprop'}
Found 34756 images belonging to 38 classes.
Found 8673 images belonging to 38 classes.
Epoch 1/2
 326/1087 ━━━━━━━━━━━━━━━━━━━━ 51:36 4s/step - accuracy: 0.2158 - loss: 3.2062

In [ ]:
best_model = load_model(best_model_path)
print("Loaded best model:", best_model_path)
